TOTO予測
===

* [データ収集](001_データ収集.ipynb)で収集したデータを使う。
* [データ加工](002_データ加工.ipynb)で加工済みとする。

In [1]:
import download
import process_data
import pandas as pd
df = download.get_db_data()
df = process_data.processing_data(df)
#display(df)

# 符号化

* チーム名・スタジアム名を符号化
* 勝ち・負け・引き分けを符号化
* 試合日を月のみ取得

In [2]:
home = pd.concat([df["ホーム"],df["アウェイ"]]).drop_duplicates().reset_index(drop=True).to_dict()
team_dict = {v:k for k, v in home.items()} # 辞書のキー・バリュー交換

In [3]:
tmp = [team_dict[df["ホーム"][i]] for i in range(len(df))]
df["home"] = tmp
tmp = [team_dict[df["アウェイ"][i]] for i in range(len(df))]
df["away"] = tmp
#display(df)

In [4]:
studium = df["スタジアム"].drop_duplicates().reset_index(drop=True).to_dict()
studium_dict = {v:k for k, v in studium.items()}

In [5]:
tmp = [studium_dict[df["スタジアム"][i]] for i in range(len(df))]
df["studium"] = tmp

勝敗はホームチームの勝ち(WIN)、負け(LOSE)、引き分け(DRAW)の三種類に符号化する。  
最終出力を獲得勝ち点で出力することを考え、WIN=3/DRAW=1/LOSE=0としておく。

カップ戦とかでPK戦の結果も含まれているが、PKは引き分けととらえることにする。

In [6]:
import re
import numpy as np
wdl = []
# win-draw-lose VゴールとかPKとかはTOTO予想に関係ないので無視
LOSE = 0
DRAW = 1
WIN = 3
OTHER = np.nan
for result in df["スコア"]:
    tmp = re.split("[-()]",result)
    if len(tmp) < 2: # X-Xという形式でないものはスルー
        wdl.append(OTHER)
        continue
    if int(tmp[0]) > int(tmp[1]):
        wdl.append(WIN)
    elif int(tmp[0]) < int(tmp[1]):
        wdl.append(LOSE)
    else:
        wdl.append(DRAW)
df["result"] = wdl
#display(df)

対戦した時期として、月の情報を入れるものとする。

試合日から情報を検出する。

In [7]:
month = []
for match_day in df["試合日"]:
    tmp = match_day.split("/")[0]
    if tmp.isdigit() == True:
        month.append(int(tmp))
    else:
        month.append(np.nan)
df["month"] = month
#display(month)

In [8]:
df = df.rename(columns={"年度":"year"})
#display(df)

使うデータを1992年~2016年に絞る。  
2017年のデータは予測の正確性検証のために別にしておくため

In [9]:
data = df[df["year"] < 2017]

結果の異常値を弾く。

異常値としてNaN(np.nan)を使ったので、NaNを含む列をドロップする。

In [10]:
data = data.dropna()

# いろんな機械学習を試す

フレームワークを使っていろいろ試してみたけど、うまくいかなかった話。

* [ディープラーニング](004_ディープラーニング.ipynb)  
ネットワークが悪いのかもしれないけど、精度21%程度にしかならなかった。
* [ベイズ分類器](005_ベイズ分類器.ipynb)  
最も精度の良かったベルヌーイ分布でも41%程度…。

# 条件付き確率で考える

単純ベイズ推定器を使ったクラスタリングをベースに、各条件における勝ち・負け・引き分けになる確率を計算し、それぞれを掛け合わせて一番高い確率のものを採用することにする。

## TOTO販売スケジュール取得

まず、確率計算する対象をTOTOの販売スケジュールから取得する

* [TOTO予定データの取得](003_TOTO予定データの取得.ipynb)より取得

In [11]:
toto, miniA, miniB, goal = download.get_toto_schedule(1048)
display(toto)

,開催日,試合開始予定時間,競技場,ホーム,アウェイ
1,10/13,15:00,長良川,岐阜,岡山
2,10/14,14:00,ニッパツ,横浜FM,鹿島
3,10/14,16:00,ＢＭＷス,湘南,柏
4,10/13,15:30,維新公園,山口,福岡
5,10/13,16:00,ＮＡＣＫ,大宮,栃木
6,10/13,15:00,ニンスタ,愛媛,横浜FC
7,10/14,14:00,Ｋｓスタ,水戸,東京Ｖ
8,10/14,14:00,石川西部,金沢,松本
9,10/14,15:00,町田,町田,大分
10,10/13,17:00,西京極,京都,徳島


## 検討する条件

1. チームの通算成績における勝率(ベースの分布？)  
$\Rightarrow \dfrac{勝利した数}{通算試合数}$
2. 対戦相手を限定した場合の勝率  
$\Rightarrow \dfrac{対戦相手に勝利した数}{対戦相手との通算試合数}$
3. 試合開催が何月かで限定した場合の勝率  
$\Rightarrow \dfrac{〇月の試合の中で勝利した数}{〇月に行われた試合数}$
4. 試合開催がどのスタジアムで限定した勝率  
$\Rightarrow \dfrac{指定スタジアムでの勝利数}{指定スタジアムでの試合数}$

In [12]:
def get_team_data(team, df):
    """
    指定したチームに関するデータのみ抽出する
    """
    team_data = df[(df["ホーム"]==team) | (df["アウェイ"]==team)]
    return team_data

def get_win_data(team, df, result=3):
    """
    指定したチームが勝利したデータのみ抽出する
    引数resultを指定すると、引き分け/負けのデータも収集可能
    """
    if result == 3:
        win_data = df[((df["ホーム"]==team) & (df["result"]==3)) | ((df["アウェイ"]==team) & (df["result"]==0))]
    elif result == 1:
        win_data = df[(df["result"]==1)]
    elif result == 0:
        win_data = df[((df["ホーム"]==team) & (df["result"]==0)) | ((df["アウェイ"]==team) & (df["result"]==3))]        
    return win_data

def get_studium_data(studium, df):
    """
    指定したスタジアムでのデータのみ抽出する
    """
    studium_data = df[(df["スタジアム"]==studium)]
    return studium_data

def get_month_data(month, df):
    """
    指定した月のデータのみ抽出する
    """
    month_data = df[df["month"]==month]
    return month_data

In [13]:
def proba_all(team, data, index="all"):
    """
    全試合に関する勝率を計算
    """
    team_data = get_team_data(team, data)
    if len(team_data)==0:
        return None
    win_data = get_win_data(team, team_data, result=3)
    draw_data = get_win_data(team, team_data, result=1)
    lose_data = get_win_data(team, team_data, result=0)
    ret = pd.DataFrame()
    ret["win"] = [len(win_data)/len(team_data)]
    ret["draw"] = [len(draw_data)/len(team_data)]
    ret["lose"] = [len(lose_data)/len(team_data)]
    #print(len(win_data),len(draw_data),len(lose_data),len(team_data))
    ret = ret.rename(index={0:index})
    return ret

def proba_oppotunity(team, oppotunity, data):
    """
    対戦相手を限定した際の勝率を計算
    """
    team_data = get_team_data(team, data)
    team_data = get_team_data(oppotunity, team_data)
    return proba_all(team, team_data,index="oppotunity")

def proba_month(team, month, data):
    """
    開催日が何月かで限定した際の勝率を計算
    """
    team_data = get_team_data(team, data)
    month_data = get_month_data(month, team_data)
    #display(month_data)
    return proba_all(team,month_data,index="month")

def proba_quarter(team, month, data):
    """
    開催月を四半期ごとに分けて勝率を計算
    Jリーグはだいたい2月終わり～12月初めまで。12月からは天皇杯。
    """
    QUARTERS = [[1,2,3],[4,5,6],[7,8,9],[10,11,12]]
    if month >= 1 or month <= 3:
        q = 0
    elif month >= 4 and month <= 6:
        q = 1
    elif month >= 7 and month <= 9:
        q = 2
    else:
        q = 3
    team_data = get_team_data(team, data)
    quarter_data = data[(data["month"]==QUARTERS[q][0]) | (data["month"]==QUARTERS[q][1]) | (data["month"]==QUARTERS[q][2])]
    #display(quarter_data)
    return proba_all(team,quarter_data,index="quarter")

def proba_studium(team, studium, data):
    """
    指定スタジアムでの試合に限定した勝率を計算
    """
    team_data = get_team_data(team, data)
    studium_data = get_studium_data(studium, team_data)
    return proba_all(team, studium_data,index="studium")

HOME = "川崎Ｆ"
AWAY = "札幌"
STUDIUM = "等々力"
tmp = pd.DataFrame()
tmp = tmp.append(proba_all(HOME, df))
tmp = tmp.append(proba_oppotunity(HOME, AWAY, df))
tmp = tmp.append(proba_month(HOME,9,df))
tmp = tmp.append(proba_studium(HOME, STUDIUM, df))
display(tmp.product().idxmax())

'win'

In [14]:
def prediction(home, away, month, studium, df):
    tmp = pd.DataFrame()
    proba = proba_all(home, df)
    if proba is not None:
        tmp = tmp.append(proba)
    proba = proba_oppotunity(home, away, df)
    if proba is not None:
        tmp = tmp.append(proba)
#    proba = proba_month(home, month, df)
#    if proba is not None:
#        tmp = tmp.append(proba)
    proba = proba_quarter(home, month, df)
    if proba is not None:
        tmp = tmp.append(proba)
    proba = proba_studium(home, studium, df)
    if proba is not None:
        tmp = tmp.append(proba)
    return tmp.product().idxmax(), tmp

In [15]:
result, pred = prediction("川崎Ｆ","札幌",9,"等々力",df)
print(result)
display(pred)

win


,win,draw,lose
all,0.510230,0.186701,0.303069
oppotunity,0.761905,0.190476,0.047619
quarter,0.384615,0.269231,0.346154
studium,0.593176,0.186352,0.220472


## アウェイチームから見た場合のデータも考える

ここまでだとホームチームから見た場合の確率しか考えていないので、アウェイチームから見た場合も考えてみる。  
確率の計算としては、単にHOME/AWAYを入れ替えるだけで同じような計算が可能。  
全く同じ方法で当てはめてみる。

In [16]:
result, pred = prediction("札幌","川崎Ｆ",9,"等々力",df)
print(result)
display(pred)

lose


,win,draw,lose
all,0.374554,0.219976,0.405470
oppotunity,0.047619,0.190476,0.761905
quarter,0.386364,0.193182,0.420455
studium,0.083333,0.083333,0.833333


## 結果の評価を行う

結果はホーム側からみた予想とアウェイ側から見た予想が双方出てくるので、下のような関係性で最終結果を出すものとする。

|アウェイ\ホーム|WIN|DRAW|LOSE|
|-------------|----|----|----|
|WIN          |DRAW|LOSE|LOSE|
|DRAW         |WIN |DRAW|LOSE|
|LOSE         |WIN |WIN |DRAW|

※最終結果はホームチームから見た結果

2016年以前のデータを使い、2017年のデータを予測して、精度を評価する。  
対象はJ1/J2とする。

In [17]:
data = df[df["year"] < 2017]
data = data.dropna()
answer = df[(df["year"] == 2017) & ((df["大会"]=="Ｊ１") | (df["大会"]=="Ｊ２"))].reset_index()
answer["result"] = answer["result"].replace(3,"win")
answer["result"] = answer["result"].replace(1,"draw")
answer["result"] = answer["result"].replace(0,"lose")
#display(answer)

In [18]:
from tqdm import tqdm_notebook
pred_home = []
pred_away = []
pred_result = []
for i in tqdm_notebook(range(len(answer))):
    tmp1, tmp2 = prediction(answer["ホーム"].iloc[i],answer["アウェイ"].iloc[i],answer["month"].iloc[i],answer["スタジアム"].iloc[i],data)
    tmp3, tmp4 = prediction(answer["アウェイ"].iloc[i],answer["ホーム"].iloc[i],answer["month"].iloc[i],answer["スタジアム"].iloc[i],data)
    pred_home.append(tmp1)
    pred_away.append(tmp3)
    if pred_home[i] == pred_away[i]:
        pred_result.append("draw")
    elif pred_home[i] == "win" or (pred_home[i] == "draw" and pred_away[i] == "lose"):
        pred_result.append("win")
    else:
        pred_result.append("lose")
        
answer["予想1"] = pred_home
answer["予想2"] = pred_away
answer["最終予想"] = pred_result
#display(answer)
print(len(answer[answer["result"]==answer["最終予想"]])/len(answer))


0.3815104166666667


In [19]:
answer

,index,year,大会,節,試合日,K/O時刻,ホーム,スコア,アウェイ,スタジアム,home,away,studium,result,month,予想1,予想2,最終予想
0,14953,2017,Ｊ１,第１節第１日,02/25(土),12:35,横浜FM,3-2,浦和,日産ス,7,2,95,win,2,win,win,draw
1,14954,2017,Ｊ１,第１節第１日,02/25(土),14:04,清水,0-1,神戸,アイスタ,5,19,6,lose,2,win,lose,win
2,14955,2017,Ｊ１,第１節第１日,02/25(土),14:04,鹿島,0-1,FC東京,カシマ,0,22,22,lose,2,win,lose,win
3,14956,2017,Ｊ１,第１節第１日,02/25(土),14:04,仙台,1-0,札幌,ユアスタ,17,18,55,win,2,win,lose,win
4,14957,2017,Ｊ１,第１節第１日,02/25(土),14:04,広島,1-1,新潟,Ｅスタ,4,24,25,draw,2,win,lose,win
5,14958,2017,Ｊ１,第１節第１日,02/25(土),14:05,鳥栖,1-3,柏,ベアスタ,16,12,53,lose,2,lose,draw,lose
6,14959,2017,Ｊ１,第１節第１日,02/25(土),15:03,Ｃ大阪,0-0,磐田,ヤンマー,13,10,52,draw,2,win,win,draw
7,14960,2017,Ｊ１,第１節第１日,02/25(土),16:03,大宮,0-2,川崎Ｆ,ＮＡＣＫ,25,20,2,lose,2,lose,win,lose
8,14961,2017,Ｊ１,第１節第２日,02/26(日),17:03,Ｇ大阪,1-1,甲府,吹田Ｓ,3,21,127,draw,2,win,lose,win
9,14962,2017,Ｊ２,第１節第１日,02/26(日),13:03,熊本,2-1,讃岐,えがおＳ,32,42,79,win,2,win,lose,win


結果、38%という数値になった。  
何も考えずに予想すると33%ですが、この場合の期待値はどうなる？

## TOTOの予定表から予測させる

TOTOの予定表から対象となるゲームのホーム・アウェイ・競技場を収集し、predictionに渡すことで結果を予想する。  

In [20]:
toto, miniA, miniB, goal = download.get_toto_schedule(1051)
#display(toto)

In [21]:
pred_home = []
pred_away = []
pred_result = []
for i in range(len(toto)):
    #print(toto["ホーム"].iloc[i],toto["アウェイ"].iloc[i],int(toto["開催日"].iloc[i].split("/")[0]),toto["競技場"].iloc[i],end="")
    tmp1, tmp2 = prediction(toto["ホーム"].iloc[i],toto["アウェイ"].iloc[i],int(toto["開催日"].iloc[i].split("/")[0]),toto["競技場"].iloc[i],df)
    tmp3, tmp4 = prediction(toto["アウェイ"].iloc[i],toto["ホーム"].iloc[i],int(toto["開催日"].iloc[i].split("/")[0]),toto["競技場"].iloc[i],df)
    #print(tmp)
    pred_home.append(tmp1)
    pred_away.append(tmp3)
    if pred_home[i] == pred_away[i]:
        pred_result.append("draw")
    elif pred_home[i] == "win" or (pred_home[i] == "draw" and pred_away[i] == "lose"):
        pred_result.append("win")
    else:
        pred_result.append("lose")
        
toto["予想1"] = pred_home
toto["予想2"] = pred_away
toto["最終予想"] = pred_result
toto

,開催日,試合開始予定時間,競技場,ホーム,アウェイ,予想1,予想2,最終予想
1,10/28,13:00,とうスタ,福島,鳥取,win,lose,win
2,10/28,14:00,長良川,岐阜,千葉,lose,win,lose
3,10/28,14:00,大銀ド,大分,松本,win,win,draw
4,10/28,14:00,デンカＳ,新潟,町田,lose,win,lose
5,10/28,14:00,鳴門大塚,徳島,横浜FC,lose,win,lose
6,10/28,14:00,ＮＤスタ,山形,水戸,win,lose,win
7,10/28,14:00,レベスタ,福岡,金沢,lose,win,lose
8,10/28,16:00,ＮＡＣＫ,大宮,京都,win,lose,win
9,10/28,17:00,ニンスタ,愛媛,東京Ｖ,lose,lose,draw
10,10/28,15:00,えがおＳ,熊本,岡山,lose,draw,lose


## スクリプト化

Bot化のためにスクリプト化したので試す。

In [3]:
import toto_predict_bayes as tp_bayes

pred_data = tp_bayes.get_prediction(1052)
display(pred_data)

,開催日,試合開始予定時間,競技場,ホーム,アウェイ,予想1,予想2,最終予想
1,11/03,14:00,ヤマハ,磐田,広島,win,lose,win
2,11/03,14:00,豊田ス,名古屋,神戸,win,win,draw
3,11/03,14:00,等々力,川崎Ｆ,柏,win,win,draw
4,11/03,14:00,日産ス,横浜FM,FC東京,win,win,draw
5,11/03,16:00,埼玉,浦和,Ｇ大阪,win,win,draw
6,11/04,14:00,札幌ド,札幌,仙台,win,lose,win
7,11/04,16:00,ベアスタ,鳥栖,長崎,lose,win,lose
8,11/04,14:00,松本,松本,東京Ｖ,win,lose,win
9,11/04,15:00,西京極,京都,愛媛,lose,lose,draw
10,11/03,14:00,デンカＳ,新潟,熊本,lose,lose,draw


# Football-Labのデータから予測

攻撃と守備は表裏一体なので、攻撃に関してはチャンス構築率、守備に関しては被チャンス構築率を見ることにし、最も値の良いチームのデータを1とした形に正規化する。  
それぞれの正規化データを掛け合わせて、チームとしての攻撃/守備がどの程度かを見てみる。  
なお、最下位チームの正規化データは0になってしまい掛け算できないので、0.1ずつ下駄をはかせてみている。

In [67]:
fldata = download.fl_get_all_data()
fldata = fldata.iloc[:,np.r_[0:2,12:22]]
#display(fldata)

In [68]:
fldata["attack"] = [float(fldata["チャンス構築率"][i].replace("%","")) for i in range(len(fldata))]
fldata["defense"] = [float(fldata["被チャンス構築率"][i].replace("%","")) for i in range(len(fldata))]

In [69]:
import scipy.stats as stats
fldata["attack"]= [(fldata["attack"][i] - min(fldata["attack"])+0.1)/(max(fldata["attack"])+0.1 - min(fldata["attack"])+0.1) for i in range(len(fldata))]
fldata["defense"]= [1.0 - (fldata["defense"][i] - min(fldata["defense"])+0.1)/(max(fldata["defense"])+0.1 - min(fldata["defense"])+0.1) for i in range(len(fldata))]

In [70]:
fldata["point"] = fldata["attack"] * fldata["defense"]

In [71]:
fldata.sort_values("point",ascending=False)

,順位,チーム,攻撃回数,シュート,チャンス構築率,ゴール,シュート成功率,被攻撃回数,被シュート,被チャンス構築率,被ゴール,被シュート成功率,attack,defense,point
0,1,川崎Ｆ,115.3,15.8,13.7%,1.6,10.1%,114.7,9.1,7.9%,0.8,8.6%,0.978261,0.984127,0.962733
5,6,浦和,114.8,13.9,12.1%,1.4,10.3%,115.4,11.1,9.6%,1.1,9.9%,0.630435,0.714286,0.450311
7,8,Ｃ大阪,122.0,14.4,11.8%,1.1,7.8%,121.5,12.7,10.4%,1.0,8.1%,0.565217,0.587302,0.331953
1,2,広島,119.4,13.9,11.6%,1.4,9.9%,119.9,12.1,10.1%,0.9,7.8%,0.521739,0.634921,0.331263
2,3,鹿島,124.2,13.8,11.1%,1.5,10.6%,122.1,11.1,9.1%,1.1,10.1%,0.413043,0.793651,0.327812
12,13,磐田,122.0,13.6,11.2%,1.1,7.8%,119.5,11.5,9.6%,1.4,11.9%,0.434783,0.714286,0.310559
10,11,横浜FM,125.3,14.7,11.7%,1.7,11.5%,121.5,12.9,10.6%,1.5,11.9%,0.543478,0.555556,0.301932
4,5,FC東京,119.4,13.2,11.0%,1.1,8.3%,121.8,11.4,9.4%,0.8,7.4%,0.391304,0.746032,0.291925
16,17,柏,126.0,14.6,11.6%,1.2,8.4%,125.6,13.7,10.9%,1.6,11.6%,0.521739,0.507937,0.265010
11,12,神戸,115.0,13.1,11.4%,1.2,8.8%,114.7,13.3,11.6%,1.4,10.8%,0.478261,0.396825,0.189786


結果、実際の順位表との乖離は多少なりあるものの、戦力的にはそこそこそれらしいのでは？という感じがする結果になった。  
15位の磐田が4番目なのは、チームとしては戦えているが決定力が低いためと思われる。逆に最下位の名古屋は決定力が高くて救われているが、あまりチームとしては戦えていないということなのかもしれない。  
なんだかんだでシュートが入るかどうか(FWの決定力に依存する？)も勝敗に非常に効くので、攻撃のほうにシュート成功率も入れた方が良いかも。

うまいこと現状のチーム状況を比較しやすい指標ができれば、そこから勝率を計算することも考えてみたいが、それだとシーズンが進まないと予想できないので、あまり意味がないかも？

# TOTOの投票をベースに予想する

Football-Labのデータを加味すると、最終的にはTOTOの投票結果に近くなるのでは？と思ったので、それがどの程度当たるのか調べてみる。

[TOTO予定データ取得](003_TOTO予定データの取得.ipynb)の際に、投票予想やくじ結果も取得していたので、このデータを使う。

In [72]:
import pandas as pd
import sqlite3
TOTO_DB = "data/toto.db"
conn = sqlite3.connect(TOTO_DB)
prize = pd.read_sql("select * from toto_prize", conn)
prize = prize.set_index("開催回",drop=True)
toto_result = pd.read_sql("select * from toto_result", conn)
conn.close()

In [73]:
toto_result = toto_result[toto_result["くじ結果"]!="中止"]
toto_result["くじ結果"] = toto_result["くじ結果"].astype("float")

## 予想通り買った場合の賞金額

投票予想通りに買った場合、賞金額はいくらになるのか調べてみた。

In [74]:
from tqdm import tqdm_notebook as tqdm
def get_expected_values(hld):
    result = toto_result[toto_result["開催回"]==hld].copy()
    if len(result)>=10:
        result[0] = result["予想0"].str.split("（",expand=True)[0].str.replace(",","").astype("int")
        result[1] = result["予想1"].str.split("（",expand=True)[0].str.replace(",","").astype("int")
        result[2] = result["予想2"].str.split("（",expand=True)[0].str.replace(",","").astype("int")
        result["予想"] = result.loc[:,[0,1,2]].idxmax(axis=1).astype("float")
        #display(result)
        #print(hld, len(result[result["くじ結果"]==result["予想"]]))
        if len(result[result["くじ結果"]==result["予想"]]) == len(result): # 全部当たり
            return prize[prize.index==str(hld)]["1等"].str.replace("円","").str.replace(",","").astype("int").values[0]
        elif len(result[result["くじ結果"]==result["予想"]]) == len(result)-1: # 一コ外れ
            return prize[prize.index==str(hld)]["2等"].str.replace("円","").str.replace(",","").astype("int").values[0]
        elif len(result[result["くじ結果"]==result["予想"]]) == len(result)-2: # 二コ外れ
            return prize[prize.index==str(hld)]["3等"].str.replace("円","").str.replace(",","").astype("int").values[0]
        else:
            return 0

In [75]:
prize_list = []
for i in tqdm(range(len(prize))):
    value = get_expected_values(int(prize.index[i]))
    if value > 0:
        prize_list.append([prize.index[i], value])

In [76]:
prize_list

[['722', 230]]

In [77]:
prize.loc["722"]

1等    107,517円
2等      1,489円
3等        230円
Name: 722, dtype: object

261~1051回までで、当たったのは3等が一回のみ。賞金額は230円。  
掛けた人が多い場合、配当も均等割される分安くなるのはある程度分かっていたものの、思った以上に当たらないことに愕然とする。